In [ ]:
!pip3 install spacy nltk pymorphy2 --quiet
!python3 -m spacy download ru_core_news_sm

2023-04-24 14:33:21.246664: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 56.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [ ]:
import nltk
import spacy
import pymorphy2
import pandas as pd

In [ ]:
nlp = spacy.load("ru_core_news_sm")
doc = nlp("Ну тут тоже, да как же, там рабочие, специально держат рабочих.")

In [ ]:
all_tokens = []
for token in doc:
    all_tokens.append(token)

In [ ]:
all_tokens

[Ну,
 тут,
 тоже,
 ,,
 да,
 как,
 же,
 ,,
 там,
 рабочие,
 ,,
 специально,
 держат,
 рабочих,
 .]

In [ ]:
df = pd.read_excel("Новая таблица.xlsx")
df = df.drop_duplicates()

df.head()

,sentence
0,"Ну тут тоже, да как же, там рабочие, специальн..."
15,"Да, милая моя, у нас, у нас жила Шарф -то, нем..."
36,"И зачем столбы, самые, с реки поставили?"
46,"Мать ни разу не помянула, и даже ни рубля не д..."
61,"Давношнее слово, розгой... били..."


In [ ]:
parsed_df = pd.read_excel("parsed_table.xlsx")
len(parsed_df)

10236

In [95]:
tokens = []
pos = []
morph = []

compound_state_1 = None
compound_state_2 = None
compound = None

quot_state_1 = None
quotation = None
for id, sentence in enumerate(df["sentence"].values):
    for token in nlp(sentence):
        if token.text == '...' or token.text == '..':  # разделить троеточия
            # print(token.text, 'троеточие')
            for ch in token.text:
                tokens.append(ch)
                pos.append(token.pos_)
                morph.append(token.morph)
        elif token.text != '-' and token.text.startswith('-'):  # убрать -то
            # print(token.text, 'убрать -то')
            text = token.text.lstrip('-')
            tokens.append('-')  # дефис
            pos.append(token.pos_)
            morph.append(token.morph)
            tokens.append(text)  # основной текст
            pos.append(token.pos_)
            morph.append(token.morph)
        elif token.text != '-' and token.text.endswith('-'):  # убрать -то
            # print(token.text, 'убрать по-')
            text = token.text.rstrip('-')
            tokens.append(text)  # основной текст
            pos.append(token.pos_)
            morph.append(token.morph)
            tokens.append('-')  # дефис
            pos.append(token.pos_)
            morph.append(token.morph)
        elif token.lemma_ in ['кто', 'что', 'какой', 'как', 'когда', 'ой']:
            # print(token.text, 'возможно, начало сложного местоимения')
            compound_state_1 = (token.text, token.pos_, token.morph)
            compound = token.text
            tokens.append(token.text)
            pos.append(token.pos_)
            morph.append(token.morph)
        elif token.text == '-' and compound_state_1:
            # print(token.text, 'дефис в сложном местоимении')
            compound = compound_state_1[0] + token.text
            compound_state_2 = (token.text, token.pos_, token.morph)
            tokens.pop()
            pos.pop()
            morph.pop()
        elif token.text in ['то', 'нибудь', 'ой'] and compound_state_1:
            # print(token.text, 'конец сложного местоимения')
            tokens.append(compound + token.text)
            pos.append(compound_state_1[1])
            morph.append(compound_state_1[2])     
            compound_state_1 = None
            compound_state_2 = None
        elif (token.text not in ['то', 'нибудь', 'ой']) and compound_state_2:
            # print(token.text, 'не конец сложного местоимения')
            tokens.append(compound_state_1[0])
            pos.append(compound_state_1[1])
            morph.append(compound_state_1[2])
            tokens.append(compound_state_2[0])
            pos.append(compound_state_2[1])
            morph.append(compound_state_2[2])
            tokens.append(token.text)
            pos.append(token.pos_)
            morph.append(token.morph)  
            compound_state_1 = None
            compound_state_2 = None
        elif len(tokens) > 0 and tokens[-1] == "'":
            tokens[-1] = tokens[-1] + token.text
            pos[-1] = token.pos_
            morph[-1] = token.morph
        else:  # остальное
            compound_state_1 = None
            compound_state_2 = None
            # print(token.text, 'остальное')
            tokens.append(token.text)
            pos.append(token.pos_)
            morph.append(token.morph)         

In [96]:
len(tokens)

10298

Выравнивание будет вручную

In [97]:
tokens = tokens[:10236]
pos = pos[:10236]
morph = morph[:10236]

In [98]:
length = abs(len(parsed_df) - len(tokens))
parsed_df['spacy_token'] = tokens
parsed_df['spacy_upos'] = pos
parsed_df['spacy_tag'] = morph

In [106]:
parsed_df[1110:1140]

,token,upos,feats,spacy_token,spacy_upos,spacy_tag
1110,Целый,ADJ,Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|N...,Целый,ADJ,"(Animacy=Inan, Case=Acc, Degree=Pos, Gender=Ma..."
1111,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,день,NOUN,"(Animacy=Inan, Case=Acc, Gender=Masc, Number=S..."
1112,лежат,VERB,Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense...,лежат,VERB,"(Aspect=Imp, Mood=Ind, Number=Plur, Person=Thi..."
1113,",",PUNCT,NaN,",",PUNCT,()
1114,а,CCONJ,NaN,а,CCONJ,()
1115,вечером,NOUN,Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing,вечером,NOUN,"(Animacy=Inan, Case=Ins, Gender=Masc, Number=S..."
1116,-,PUNCT,NaN,-,PART,()
1117,идем,VERB,Aspect=Imp|Mood=Ind|Number=Plur|Person=1|Tense...,идем,VERB,"(Aspect=Imp, Mood=Ind, Number=Plur, Person=Fir..."
1118,.,PUNCT,NaN,.,PUNCT,()
1119,А,CCONJ,NaN,А,CCONJ,()


In [92]:
parsed_df.to_excel("parsed_table_spacy.xlsx", index=False)